### 上一小节，我们通过示例，主要讲解了如何使用pandas对单表进行数据分析，统计等操作.

### 本节，我们将通过另一个示例，讲解多表连接后，对数据进行清洗，分析等操作.

### 示例数据采用美国各州人口统计数据，一共有三张统计数据表分别为：
- state-abbrevs.csv 
- state-population.csv 
- state-areas.csv

- 本次分享的主要任务包括以下几个方面：
    - 导入文件，查看原始数据
    - 将人口数据和各州简称数据进行合并
    - 将合并的数据中重复的abbreviation列进行删除
    - 查看存在缺失数据的列
    - 找到有哪些state/region使得state的值为NaN，进行去重操作
    - 为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN
    - 合并各州面积数据areas
    - 我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行
    - 去除含有缺失数据的行
    - 找出2010年的全民人口数据
    - 计算各州的人口密度
    - 排序，并找出人口密度最高的五个州

In [1]:
# 导入必要的包
import numpy as np
import pandas as pd

#### 导入文件，查看原始数据

In [2]:
# 分别读取三个csv文件
abb = pd.read_csv('./state-abbrevs.csv')
pop = pd.read_csv('./state-population.csv')
area = pd.read_csv('./state-areas.csv')

In [3]:
abb.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [4]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [5]:
area.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


#### 将人口数据和各州简称数据进行合并

#### 将合并的数据中重复的abbreviation列进行删除

#### 查看存在缺失数据的列

#### 找到有哪些state/region使得state的值为NaN，进行去重操作(哪些简称对应的全程值为空，且对符合要求的简称进行去重)

In [6]:
# 找出state的空对应的简称数据


In [7]:
# 对符合要求的简称进行去重


#### 为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN
    - PR和USA对应的全称的值为NAN

In [8]:
# 找出符合要求行数据的行索引

# 将indexs对应的行中的state列的空值批量赋值成Puerto Rico


In [9]:
# 对简称为USA的行采用同样的操作，去除空值


#### 合并各州面积数据areas

In [10]:
# show abb_pop


In [11]:
# show area


#### 我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行

#### 去除含有缺失数据的行

#### 找出2010年的全民人口数据

In [12]:
# 找到2010年的数据，再找total的数据


In [13]:
# 使用牛逼的query


#### 赠送内容：把索引变为日期时间类型

In [14]:
# abb_pop_area.index.year
# type(abb_pop_area.index.year)

#### 计算各州的人口密度

#### 排序，并找出2013年人口密度最高的五个州

In [15]:
# 获取2013年各州人口总数数据信息


In [16]:
# 2013年人口密度最高的五个州


In [17]:
# set state as index


In [18]:
# show result as a dictionary
